In [48]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
import numpy as np
from scipy import stats

In [2]:
#Read Osun crosschecked csv into dataframe
Osun_all = pd.read_csv('OSUN_crosschecked.csv')

#Dusplay data
Osun_all

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,Result_Sheet_Corrected,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File
0,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-001,TOWN HALL IWARA,181,942,True,-1,False,False,False,False,UNKNOWN,44,273,0,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....
1,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-003,"L.A. SCHOOL, IWIKUN",245,881,True,-1,False,False,False,False,UNKNOWN,40,11,175,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....
2,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-004,"METHODIST PRY. SCHOOL, AYETORO",117,308,True,-1,False,False,False,False,UNKNOWN,14,3,99,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....
3,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-007,OPEN SPACE ODO-IWARA,187,538,True,-1,False,False,False,False,UNKNOWN,25,4,155,0,https://docs.inecelectionresults.net/elections...
4,OSUN,ATAKUMOSA EAST,IGANGAN,29-01-02-001,"N.U.D PRY. SCHOL, IGANGAN",187,558,True,-1,False,False,False,False,UNKNOWN,22,0,161,0,https://docs.inecelectionresults.net/elections...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2243,OSUN,OSOGBO,ATAOJA 'E',29-30-05-042,"OPEN SPACE, SALVATION ARMY PRY.SCHOOL CAR PARK...",157,546,True,-1,False,False,False,False,UNKNOWN,25,4,14,2,https://docs.inecelectionresults.net/elections...
2244,OSUN,OSOGBO,ATAOJA 'E',29-30-11-019,"OPEN SPACE, SALVATION ARMY PRY.SCHOOL CAR PARK...",157,546,True,-1,False,False,False,False,UNKNOWN,65,19,56,0,https://docs.inecelectionresults.net/elections...
2245,OSUN,OSOGBO,BABA KEKERE,29-30-14-019,"OPEN SPACE, BOOSERE STREET JUNCTION, AROGUNMAS...",139,179,True,-1,False,False,False,False,UNKNOWN,46,4,50,1,https://docs.inecelectionresults.net/elections...
2246,OSUN,OSOGBO,BABA KEKERE,29-30-14-020,"OPEN SPACE, BOOSERE STREET JUNCTION, AROGUNMAS...",139,179,True,-1,False,True,False,False,UNKNOWN,25,5,44,1,https://docs.inecelectionresults.net/elections...


In [3]:
#Check for any Null values in the Osun_all 
Osun_all[Osun_all.isna().any(axis=1)]

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,Result_Sheet_Corrected,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File


In [4]:
#Read Osun polling unit longitute and latitude csv into dataframe
Osun_polling_unit = pd.read_csv('polling_unit_osun.csv')

#Display data
Osun_polling_unit

,PU-Name,ward_name,local_government_name,state_name,location.latitude,location.longitude
0,ASIPA OPEN SPACE,ILEOGBO IV,AYEDIRE,OSUN,7.330648,3.849630
1,"AFRICAN CHURCH AREA, OPEN SPACE",ILEOGBO IV,AYEDIRE,OSUN,7.599449,4.280376
2,GBOGBO OPEN SPACE,ILEOGBO IV,AYEDIRE,OSUN,7.617492,4.267115
3,AYI OPEN SPACE,ILEOGBO IV,AYEDIRE,OSUN,7.768887,4.540383
4,OPEN SPACE ASIPA II,ILEOGBO IV,AYEDIRE,OSUN,NaN,NaN
...,...,...,...,...,...,...
3758,"OPEN SPACE, YARO JUNCTION, IREE",ISALE ASA IREE,BORIPE,OSUN,7.923520,4.724540
3759,"OPEN SPACE,OPP. SURULERE HOUSE ALONG BINUYO S...",ISALE ASA IREE,BORIPE,OSUN,NaN,NaN
3760,"OPEN SPACE, OSUN STATE POLYTECHNIC GATE",ISALE ASA IREE,BORIPE,OSUN,7.923520,4.724540
3761,"OPEN SPACE, OPPOSITE SAOYAD FILLING STATION, IREE",ISALE ASA IREE,BORIPE,OSUN,7.933784,4.715262


In [5]:
# Merge polling_unit_osun.csv with OSUN_crosschecked.csv using left join
New_Osun_crosschecked = pd.merge(Osun_all, Osun_polling_unit, on='PU-Name', how='left')

# query to identify rows where PU-Name is in Osun_all but not in Osun_polling_unit
query = New_Osun_crosschecked['PU-Name'].isin(Osun_all['PU-Name']) & ~New_Osun_crosschecked['PU-Name'].isin(Osun_polling_unit['PU-Name'])
New_Osun_crosschecked = New_Osun_crosschecked[~query]

# Drop redundant columns
Osun_polling = New_Osun_crosschecked.drop(['ward_name', 'local_government_name', 'state_name'], axis=1)

# Display data
Osun_polling

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,location.latitude,location.longitude
0,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-001,TOWN HALL IWARA,181,942,True,-1,False,...,False,False,UNKNOWN,44,273,0,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,7.466182,4.678075
1,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-003,"L.A. SCHOOL, IWIKUN",245,881,True,-1,False,...,False,False,UNKNOWN,40,11,175,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,7.586975,4.629450
2,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-004,"METHODIST PRY. SCHOOL, AYETORO",117,308,True,-1,False,...,False,False,UNKNOWN,14,3,99,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,7.846332,4.487902
3,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-007,OPEN SPACE ODO-IWARA,187,538,True,-1,False,...,False,False,UNKNOWN,25,4,155,0,https://docs.inecelectionresults.net/elections...,7.540318,4.825199
4,OSUN,ATAKUMOSA EAST,IGANGAN,29-01-02-001,"N.U.D PRY. SCHOL, IGANGAN",187,558,True,-1,False,...,False,False,UNKNOWN,22,0,161,0,https://docs.inecelectionresults.net/elections...,7.546745,3.788141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312,OSUN,OSOGBO,ATAOJA 'E',29-30-05-042,"OPEN SPACE, SALVATION ARMY PRY.SCHOOL CAR PARK...",157,546,True,-1,False,...,False,False,UNKNOWN,25,4,14,2,https://docs.inecelectionresults.net/elections...,7.781225,4.545717
2313,OSUN,OSOGBO,ATAOJA 'E',29-30-11-019,"OPEN SPACE, SALVATION ARMY PRY.SCHOOL CAR PARK...",157,546,True,-1,False,...,False,False,UNKNOWN,65,19,56,0,https://docs.inecelectionresults.net/elections...,7.781225,4.545717
2314,OSUN,OSOGBO,BABA KEKERE,29-30-14-019,"OPEN SPACE, BOOSERE STREET JUNCTION, AROGUNMAS...",139,179,True,-1,False,...,False,False,UNKNOWN,46,4,50,1,https://docs.inecelectionresults.net/elections...,7.786754,4.557996
2315,OSUN,OSOGBO,BABA KEKERE,29-30-14-020,"OPEN SPACE, BOOSERE STREET JUNCTION, AROGUNMAS...",139,179,True,-1,False,...,False,False,UNKNOWN,25,5,44,1,https://docs.inecelectionresults.net/elections...,7.786754,4.557996


In [6]:
#Export Null locations
Osun_polling[Osun_polling.isna().any(axis=1)].to_csv('Osun_polling_null.csv', index=False)

#Remove rows with null values from the dataframe
Osun_polling_no_null = Osun_polling.dropna()

#Reset index
Osun_polling_no_null = Osun_polling_no_null.reset_index(drop=True)

#Display data
Osun_polling_no_null

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,location.latitude,location.longitude
0,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-001,TOWN HALL IWARA,181,942,True,-1,False,...,False,False,UNKNOWN,44,273,0,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,7.466182,4.678075
1,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-003,"L.A. SCHOOL, IWIKUN",245,881,True,-1,False,...,False,False,UNKNOWN,40,11,175,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,7.586975,4.629450
2,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-004,"METHODIST PRY. SCHOOL, AYETORO",117,308,True,-1,False,...,False,False,UNKNOWN,14,3,99,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,7.846332,4.487902
3,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-007,OPEN SPACE ODO-IWARA,187,538,True,-1,False,...,False,False,UNKNOWN,25,4,155,0,https://docs.inecelectionresults.net/elections...,7.540318,4.825199
4,OSUN,ATAKUMOSA EAST,IGANGAN,29-01-02-001,"N.U.D PRY. SCHOL, IGANGAN",187,558,True,-1,False,...,False,False,UNKNOWN,22,0,161,0,https://docs.inecelectionresults.net/elections...,7.546745,3.788141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,OSUN,OSOGBO,ATAOJA 'E',29-30-05-042,"OPEN SPACE, SALVATION ARMY PRY.SCHOOL CAR PARK...",157,546,True,-1,False,...,False,False,UNKNOWN,25,4,14,2,https://docs.inecelectionresults.net/elections...,7.781225,4.545717
2012,OSUN,OSOGBO,ATAOJA 'E',29-30-11-019,"OPEN SPACE, SALVATION ARMY PRY.SCHOOL CAR PARK...",157,546,True,-1,False,...,False,False,UNKNOWN,65,19,56,0,https://docs.inecelectionresults.net/elections...,7.781225,4.545717
2013,OSUN,OSOGBO,BABA KEKERE,29-30-14-019,"OPEN SPACE, BOOSERE STREET JUNCTION, AROGUNMAS...",139,179,True,-1,False,...,False,False,UNKNOWN,46,4,50,1,https://docs.inecelectionresults.net/elections...,7.786754,4.557996
2014,OSUN,OSOGBO,BABA KEKERE,29-30-14-020,"OPEN SPACE, BOOSERE STREET JUNCTION, AROGUNMAS...",139,179,True,-1,False,...,False,False,UNKNOWN,25,5,44,1,https://docs.inecelectionresults.net/elections...,7.786754,4.557996


In [7]:
#Read the geocode file of the previous null values csv into dataframe
Osun_api_geo = pd.read_csv('Osun_api_fixed.csv')

#Find rows with duplicate PU-Name
duplicate_pu = Osun_api_geo[Osun_api_geo['PU-Name'].duplicated(keep=False)]

#Access data
Osun_api_geo

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,location.latitude,location.longitude,state
0,OSUN,ATAKUMOSA EAST,IPOLE,29-01-03-003,ST. ANDREW ISE-IJESA,243,565,True,-1,False,...,False,UNKNOWN,84,3,129,0,https://docs.inecelectionresults.net/elections...,7.630560,4.782640,Osun State
1,OSUN,ATAKUMOSA EAST,FOREST RESERVE 1,29-01-07-003,SOKOTO I,213,591,True,-1,False,...,False,UNKNOWN,80,2,130,0,https://docs.inecelectionresults.net/elections...,7.810490,4.528880,Osun State
2,OSUN,ATAKUMOSA EAST,FOREST RESERVE 1,29-01-07-005,SOKOTO S.B OJO,143,312,True,-1,False,...,False,UNKNOWN,68,0,75,0,https://docs.inecelectionresults.net/elections...,7.407087,3.902309,Osun State
3,OSUN,ATAKUMOSA EAST,FOREST RESERVE 1,29-01-07-006,SOKOTO S.B OJO,143,312,True,-1,False,...,False,UNKNOWN,155,3,102,0,https://docs.inecelectionresults.net/elections...,7.407087,3.902309,Osun State
4,OSUN,ATAKUMOSA EAST,FOREST RESERVE 1,29-01-07-007,OWADA,70,170,True,-1,False,...,False,UNKNOWN,34,1,34,0,https://docs.inecelectionresults.net/elections...,7.810490,4.528880,Osun State
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,OSUN,IFE CENTRAL,IREMO/AJEBANDELE,29-11-05-016,COMMUNITY SCH. KAJOLA AKILE,54,79,True,-1,False,...,False,UNKNOWN,23,17,9,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,7.552956,4.458804,Osun State
261,OSUN,IFE CENTRAL,IREMO/AJEBANDELE,29-11-05-025,HOLY SAVIOUR PRY SCH FASINA,73,92,True,-1,False,...,False,UNKNOWN,44,12,12,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,NaN,NaN,NaN
262,OSUN,IFE CENTRAL,IREMO/AJEBANDELE,29-11-05-027,OPEN SPACE BASKET BALL COURT BLOCK 3,44,371,True,-1,False,...,False,UNKNOWN,11,26,4,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,NaN,NaN,NaN
263,OSUN,ILESA WEST,OMOFE/IDASA,29-19-04-011,"OPEN SPACE OPPOSITE ILE AGBEDE, IDASA, ILESA",96,171,True,-1,False,...,False,UNKNOWN,30,4,50,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,4.876738,6.912980,Rivers


In [8]:
#Drop rows with wrong states == wrong lat and long a and null values
Osun_api_cleaned = Osun_api_geo[
    ((Osun_api_geo['state'] == 'Osun') | (Osun_api_geo['state'] == 'Osun State')) & 
    (Osun_api_geo['location.latitude'].notna()) & 
    (Osun_api_geo['location.longitude'].notna())
]
#Drop state column
Osun_api_cleaned = Osun_api_cleaned.drop(['state'], axis=1)

Osun_api_cleaned

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,location.latitude,location.longitude
0,OSUN,ATAKUMOSA EAST,IPOLE,29-01-03-003,ST. ANDREW ISE-IJESA,243,565,True,-1,False,...,False,False,UNKNOWN,84,3,129,0,https://docs.inecelectionresults.net/elections...,7.630560,4.782640
1,OSUN,ATAKUMOSA EAST,FOREST RESERVE 1,29-01-07-003,SOKOTO I,213,591,True,-1,False,...,False,False,UNKNOWN,80,2,130,0,https://docs.inecelectionresults.net/elections...,7.810490,4.528880
2,OSUN,ATAKUMOSA EAST,FOREST RESERVE 1,29-01-07-005,SOKOTO S.B OJO,143,312,True,-1,False,...,False,False,UNKNOWN,68,0,75,0,https://docs.inecelectionresults.net/elections...,7.407087,3.902309
3,OSUN,ATAKUMOSA EAST,FOREST RESERVE 1,29-01-07-006,SOKOTO S.B OJO,143,312,True,-1,False,...,False,False,UNKNOWN,155,3,102,0,https://docs.inecelectionresults.net/elections...,7.407087,3.902309
4,OSUN,ATAKUMOSA EAST,FOREST RESERVE 1,29-01-07-007,OWADA,70,170,True,-1,False,...,False,False,UNKNOWN,34,1,34,0,https://docs.inecelectionresults.net/elections...,7.810490,4.528880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,OSUN,OSOGBO,EKETA,29-30-13-005,"44, OLUSEGUN STREET",251,642,True,-1,False,...,False,False,UNKNOWN,173,2,71,0,https://docs.inecelectionresults.net/elections...,7.779167,4.553889
257,OSUN,OSOGBO,EKETA,29-30-13-012,"8, KUJENYO STREET",135,233,True,-1,False,...,False,False,UNKNOWN,80,3,50,0,https://docs.inecelectionresults.net/elections...,7.779167,4.553889
258,OSUN,AYEDIRE,OLUPONNA 1,29-04-01-006,OLODE OPEN SPACE,66,84,True,-1,False,...,False,False,UNKNOWN,142,1,149,0,https://docs.inecelectionresults.net/elections...,7.724798,4.482577
259,OSUN,BORIPE,ISALE-OYO,29-06-04-009,"OPEN SPACE, AJEGUNLE JUNCTION",120,158,True,-1,False,...,False,False,UNKNOWN,47,3,23,0,https://docs.inecelectionresults.net/elections...,7.818669,4.637061


In [22]:
#Add rows from Osun_api_cleaned to Osun_polling_no_null
Osun_merged = pd.concat([Osun_polling_no_null, Osun_api_cleaned], axis=0, ignore_index=True)

#Arrange Alphabetically
Osun_merged = Osun_merged.sort_values('LGA')
Ex=Osun_merged

#Export Data
Ex = Osun_merged.to_csv('Osun_lat_long.csv', index=False)

#Display
Osun_merged

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,location.latitude,location.longitude
0,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-001,TOWN HALL IWARA,181,942,True,-1,False,...,False,False,UNKNOWN,44,273,0,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,7.466182,4.678075
25,OSUN,ATAKUMOSA EAST,AYEGUNLE,29-01-06-002,"L.A. PRY. SCHOOL, AJIDO",221,667,True,-1,False,...,False,False,UNKNOWN,71,0,144,0,https://docs.inecelectionresults.net/elections...,7.653830,4.833019
26,OSUN,ATAKUMOSA EAST,AYEGUNLE,29-01-06-004,C.A.C. PRY. SCHOOL AKANNI,84,379,True,-1,False,...,False,False,UNKNOWN,3,0,78,0,https://docs.inecelectionresults.net/elections...,7.615625,4.749116
27,OSUN,ATAKUMOSA EAST,AYEGUNLE,29-01-06-006,ITAMERIN,95,400,True,-1,True,...,False,False,UNKNOWN,29,0,60,0,https://docs.inecelectionresults.net/elections...,7.506388,4.826412
28,OSUN,ATAKUMOSA EAST,FOREST RESERVE II,29-01-08-002,GRAMMAR SCHOOL ALAKA,252,619,True,-1,False,...,False,False,UNKNOWN,153,0,95,0,https://docs.inecelectionresults.net/elections...,5.543938,6.238907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868,OSUN,OSOGBO,ATAOJA 'E',29-30-05-005,OGIDAN PRY SCHOOL,222,750,True,-1,False,...,False,False,UNKNOWN,103,23,71,0,https://docs.inecelectionresults.net/elections...,7.767637,4.585292
1867,OSUN,OSOGBO,ATAOJA 'E',29-30-05-004,OGIDAN PRY SCHOOL,222,750,True,-1,False,...,False,False,UNKNOWN,113,10,91,0,https://docs.inecelectionresults.net/elections...,7.767637,4.585292
1866,OSUN,OSOGBO,ATAOJA 'E',29-30-05-002,OSOGBO GRAMMAR SCHOOL,293,952,True,-1,False,...,False,False,UNKNOWN,111,61,114,0,https://docs.inecelectionresults.net/elections...,7.781357,4.537947
1864,OSUN,OSOGBO,ATAOJA 'D',29-30-04-022,ALHAJI WOLEOLA JUNCTION II,274,989,True,-1,False,...,False,False,UNKNOWN,126,27,101,0,https://docs.inecelectionresults.net/elections...,7.767840,4.528572


In [30]:
#Formula for haversine_distance
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  #Earth's radius(Km)
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance


In [79]:
#Maximum distance to be considered as a neighbout being 2km
def find_neighbors(df, max_distance=2):
    if df is None or not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a valid pandas DataFrame")
    
    if 'location.latitude' not in df.columns or 'location.longitude' not in df.columns:
        raise ValueError("DataFrame must contain 'location.latitude' and 'location.longitude' columns")
    
    neighbors = []
    for i, row in df.iterrows():
        neighbor_list = []
        for j, other_row in df.iterrows():
            if i != j:
                try:
                    dist = haversine_distance(row['location.latitude'], row['location.longitude'],
                                              other_row['location.latitude'], other_row['location.longitude'])
                    if dist <= max_distance:
                        neighbor_info = f"{other_row['PU-Name']}, {other_row['Ward']}, {other_row['LGA']}"
                        neighbor_list.append(neighbor_info)
                except KeyError as e:
                    print(f"Error accessing data for row {i}: {e}")
                except Exception as e:
                    print(f"Unexpected error for row {i}: {e}")
        neighbors.append('; '.join(neighbor_list))
    return neighbors

try:
    Osun_merged['Neighbours'] = find_neighbors(Osun_merged)
except Exception as e:
    print(f"Error creating 'Neighbours' column: {e}")

KeyboardInterrupt: 

In [81]:
Osun_merged['Neighbours'] = find_neighbors(Osun_merged)

# Display
Osun_merged

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,location.latitude,location.longitude,Neighbours,APC_outlier_score,LP_outlier_score,PDP_outlier_score,NNPP_outlier_score,composite_outlier_score,outlier_details,neighbors_list
0,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-001,TOWN HALL IWARA,181,942,True,-1,False,...,7.466182,4.678075,"AGBABIAKA VILLAGE, IFEWARA I, ATAKUMOSA WEST;...",0,0,0,0,0,,"[TOWN HALL, IFEWARA, IFEWARA I, ATAKUMOSA WES..."
1245,OSUN,IREPODUN,ELERIN 'B',29-20-08-010,AGBEDE AREA,167,359,True,-1,False,...,7.839958,4.478847,"OLUWOMI QUARTERS I, OLOBU 'B', IREPODUN; OKE-K...",0,0,0,0,0,,"[METHODIST PRY. SCHOOL, AYETORO, IWARA, ATAKUM..."
1352,OSUN,IREWOLE,IKIRE 'K',29-21-11-020,"EFUNLE ALAGUNTAN PRY. SCHOOL, EFUNLE ALAGUNTAN",97,213,True,-1,False,...,7.367829,4.201334,"BAPTIST DAY SCHOOL, ONIKAN VILLAGE, IKIRE 'K',...",0,0,0,0,0,,"[ABEWELA VILLAGE, ABEWELA VILLAGE, OBALUFON, A..."
1351,OSUN,IREWOLE,IKIRE 'K',29-21-11-019,"ITAMERIN VILLAGE, ITAMERIN",188,373,True,-1,False,...,7.372267,4.183420,"BAPTIST DAY SCHOOL, ONIKAN VILLAGE, IKIRE 'K',...",0,0,0,0,0,,"[ABEWELA VILLAGE, ABEWELA VILLAGE, OBALUFON, A..."
1350,OSUN,IREWOLE,IKIRE 'K',29-21-11-018,"BAPTIST DAY SCHOOL, ONIKAN VILLAGE",142,292,True,-1,False,...,7.370015,4.187199,"EFUNLE ALAGUNTAN PRY. SCHOOL, EFUNLE ALAGUNTAN...",0,0,0,0,0,,"[ABEWELA VILLAGE, ABEWELA VILLAGE, OBALUFON, A..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,OSUN,IFE CENTRAL,AKARABATA,29-11-10-004,FRONTAGE OF L.I.E.'S OFFICE,149,651,True,-1,False,...,7.495976,4.547402,"ST. MARY'S PRY. SCHOOL, ORANRAN I, ORANRAN WAR...",0,0,0,0,0,,"[ST. DAVID.S PRIMARY SCHOOL, OBADA AREA, ODEOM..."
611,OSUN,IFE CENTRAL,AKARABATA,29-11-10-005,URBAN DAY GRAMMAR SCHOOL,93,581,True,-1,False,...,7.488503,4.532230,"ST. DAVID.S PRIMARY SCHOOL, OBADA AREA, ODEOMU...",0,0,0,0,0,,"[ST. DAVID.S PRIMARY SCHOOL, OBADA AREA, ODEOM..."
694,OSUN,IFE EAST,MOORE,29-13-01-008,"ABA OMPETU, OMPETU VILLAGE",69,227,True,-1,False,...,7.480449,4.572070,"ST. MARY'S PRY. SCHOOL, ORANRAN I, ORANRAN WAR...",0,0,0,0,0,,"[IGBIRA ISALE AREA, IGBIRA VILLAGE ODE OMU, IJ..."
695,OSUN,IFE EAST,MOORE,29-13-01-010,"OPA AREA, ILESHA ROAD",428,1408,True,-1,False,...,7.547977,4.583901,,0,0,0,0,0,,[]


In [83]:
# neighbour report
neighbor_counts = Osun_merged['Neighbours'].apply(lambda x: len(x.split(';')) if x else 0)
print("\nNeighbor statistics:")
print(f"Average number of neighbors: {neighbor_counts.mean():.2f}")
print(f"Maximum number of neighbors: {neighbor_counts.max()}")
print(f"Minimum number of neighbors: {neighbor_counts.min()}")
print(f"Polling units with no neighbors: {(neighbor_counts == 0).sum()}")



Neighbor statistics:
Average number of neighbors: 54.03
Maximum number of neighbors: 241
Minimum number of neighbors: 0
Polling units with no neighbors: 153


In [ ]:
def extract_pu_names(neighbors_string):
    return [name.split(',')[0].strip() for name in neighbors_string.split(';')]

In [ ]:
def calculate_outlier_scores(df):
    parties = ['APC', 'LP', 'PDP', 'NNPP']
    
    # Extract PU-Names from Neighbours column
    df['neighbor_pu_names'] = df['Neighbours'].apply(extract_pu_names)
    
    for party in parties:
        df[f'{party}_outlier_score'] = df.apply(lambda row: calculate_party_outlier_score(row, df, party), axis=1)

    df['composite_outlier_score'] = df[[f'{party}_outlier_score' for party in parties]].max(axis=1)
    
    outlier_details = []
    for party in parties:
        outlier_details.append(df[party].astype(str) + ' (' + df[f'{party}_outlier_score'].round(2).astype(str) + ')')
    df['outlier_details'] = pd.DataFrame(outlier_details).T.agg(', '.join, axis=1)

    return df

In [ ]:
def calculate_party_outlier_score(row, df, party):
    neighbor_votes = df[df['PU-Name'].isin(row['neighbor_pu_names'])][party]
    if len(neighbor_votes) == 0:
        return 0
    
    mean_votes = neighbor_votes.mean()
    std_votes = neighbor_votes.std()
    
    if std_votes == 0:
        return 10 if row[party] != mean_votes else 0
    
    z_score = (row[party] - mean_votes) / std_votes
    return abs(z_score)


In [97]:
Osun_merged = calculate_outlier_scores(Osun_merged)


Osun_merged = Osun_merged.sort_values('composite_outlier_score', ascending=False)


Osun_merged.to_csv('Osun_polling_sorted_outlier_scores.csv', index=False)
Osun_merged

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,location.longitude,Neighbours,APC_outlier_score,LP_outlier_score,PDP_outlier_score,NNPP_outlier_score,composite_outlier_score,outlier_details,neighbors_list,neighbor_pu_names
378,OSUN,EDE SOUTH,ALAJUE I,29-08-04-006,ODE-OKE,335,833,True,-1,False,...,4.450365,"L.A. PRY. SCHOOL, ALUSEKERE, ALUSEKERE, EDE NO...",0.601742,0.084387,1.810891,311.293632,311.293632,"113 (0.6), 3 (0.08), 0 (1.81), 207 (311.29)","[L.A. PRY. SCHOOL, KAJOLA, OKE BODE, ATAKUMOSA...","[L.A. PRY. SCHOOL, ALUSEKERE JUNCTION, ISEKI O..."
973,OSUN,ILA,ISEDO I,29-17-04-001,OLOPONDA I,198,370,True,-1,False,...,4.913126,"IGBONNIBI TOWN HALL II, EJIGBO I, ILA; BAPTIS...",0.059271,0.273301,1.683438,277.804476,277.804476,"87 (0.06), 2 (0.27), 0 (1.68), 106 (277.8)","[BAPTIST PRY. SCHOOL I, OBAALE, IFEDAYO, BAPT...","[IGBONNIBI TOWN HALL II, BAPTIST PRY. SCHOOL I..."
333,OSUN,EDE NORTH,ALUSEKERE,29-07-05-004,"OWODE COMM. PRY SCHOOL,",771,2543,True,-1,False,...,4.486007,"OPEN SPACE BY OWODE MARKET SQUARE, COLLEGE/EGB...",3.358757,207.182287,4.740234,10.000000,207.182287,"217 (3.36), 147 (207.18), 380 (4.74), 2 (10.0)","[OLODE OPEN SPACE, OLUPONNA 1, AYEDIRE, OPEN...","[OPEN SPACE BY OWODE MARKET SQUARE, OLODE OPEN..."
346,OSUN,EDE NORTH,SABO/AGBONGBE II,29-07-07-001,BESIDE EBELOMO IND.,194,750,True,-1,False,...,4.437060,"L.A. PRY. SCHOOL, ALUSEKERE, ALUSEKERE, EDE NO...",0.299847,2.045248,1.957596,171.196537,171.196537,"50 (0.3), 14 (2.05), 0 (1.96), 127 (171.2)","[PERE LANE I, ISIBO/BUARI-ISOLA, EDE NORTH, B...","[L.A. PRY. SCHOOL, ALUSEKERE JUNCTION, ISEKI O..."
1916,OSUN,OSOGBO,JAGUN 'A',29-30-09-004,"24, OLUGUNNA STREET`",196,443,True,-1,False,...,4.562977,"AKUMORA, AWALA II, ISOKAN; ERUKU HALL I, OOSA...",0.604801,0.705792,2.007750,154.775824,154.775824,"144 (0.6), 0 (0.71), 0 (2.01), 49 (154.78)","[ST. JAMES PRY. SCHOOL, ILAA, MUROKO, ATAKUMOS...","[AKUMORA, ERUKU HALL I, KOLAWOLE JUNCTION, OWO..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,OSUN,EJIGBO,ILAWO/ISOKO/ISUNDUNRIN,29-10-09-010,ISOKO MARKET STALL,279,603,True,-1,True,...,4.271879,"OOSA/JAGUN/ARO, ILAWO/ISOKO/ISUNDUNRIN, EJIGBO",NaN,NaN,NaN,NaN,NaN,"97 (nan), 0 (nan), 176 (nan), 0 (nan)","[OOSA/JAGUN/ARO, ILAWO/ISOKO/ISUNDUNRIN, EJIGBO]",[OOSA/JAGUN/ARO]
508,OSUN,EJIGBO,INISA II/AFAAKE/AYEGUNLE,29-10-11-009,ADO-ORI-OKE,138,250,True,-1,False,...,4.264744,"AWOKEKERE VILLAGE, OKE-OBA I, IWO",NaN,NaN,NaN,NaN,NaN,"79 (nan), 0 (nan), 53 (nan), 0 (nan)","[AWOKEKERE VILLAGE, OKE-OBA I, IWO]",[AWOKEKERE VILLAGE]
485,OSUN,EJIGBO,IFEODAN 'A'/OWU-ILE,29-10-07-004,"OKE AFIN/IWO ATE, IFE-ODAN",315,833,True,-1,False,...,4.105331,"IFE-ODAN POSTAL AGENCY II, IFEODAN 'A'/OWU-IL...",NaN,NaN,NaN,NaN,NaN,"136 (nan), 0 (nan), 175 (nan), 0 (nan)","[OKE-OWU HALL I, OKE OWU AREA ODEOMU, IJEGBE/O...","[IFE-ODAN POSTAL AGENCY II, OKE-OWU HALL I, O..."
490,OSUN,EJIGBO,ILAWO/ISOKO/ISUNDUNRIN,29-10-09-002,OOSA/JAGUN/ARO,273,515,True,-1,True,...,4.277108,"ISOKO MARKET STALL, ILAWO/ISOKO/ISUNDUNRIN, EJ...",NaN,NaN,NaN,NaN,NaN,"159 (nan), 0 (nan), 105 (nan), 0 (nan)","[ISOKO MARKET STALL, ILAWO/ISOKO/ISUNDUNRIN, E...",[ISOKO MARKET STALL]


In [100]:
# 10 most anomalous polling units
print("Top 10 Most anomaly Polling Units:")
columns_to_display = ['PU-Name', 'LGA', 'APC', 'LP', 'PDP', 'NNPP', 'composite_outlier_score', 'outlier_details']
print(Osun_merged[columns_to_display].head(10))

#report
print("\nSummary Report:")
print("===============")
for party in ['APC', 'LP', 'PDP', 'NNPP']:
    print(f"\n{party} Outlier Statistics:")
    print(f"Average Outlier Score: {Osun_merged[f'{party}_outlier_score'].mean():.2f}")
    print(f"Median Outlier Score: {Osun_merged[f'{party}_outlier_score'].median():.2f}")
    print(f"Standard Deviation of Outlier Score: {Osun_merged[f'{party}_outlier_score'].std():.2f}")

Top 10 Most anomaly Polling Units:
                                 PU-Name             LGA  APC   LP  PDP  NNPP  \
378                              ODE-OKE       EDE SOUTH  113    3    0   207   
973                           OLOPONDA I             ILA   87    2    0   106   
333              OWODE COMM. PRY SCHOOL,       EDE NORTH  217  147  380     2   
346                  BESIDE EBELOMO IND.       EDE NORTH   50   14    0   127   
1916                24, OLUGUNNA STREET`          OSOGBO  144    0    0    49   
1686               OLATUNJI AJAYI STREET        OLORUNDA  166   24  101    22   
0                        TOWN HALL IWARA  ATAKUMOSA EAST   44  273    0     0   
619                 IFE CITY HALL AREA I     IFE CENTRAL   90    5    0    35   
1566  ST. MICHAEL'S PRY. SCHOOL, OKUKU I        ODO-OTIN   37   23   72     0   
293            POLY. STAFF. SCHOOL, IREE          BORIPE    8  120    2     0   

      composite_outlier_score                                 outlier_det